# Compile MOD files

Before validating cell models, compile their channel .mod files by running:

`./compile_mod.sh`

from the 

`[repo]/prev_ob_models` directory

# Imports

In [2]:
import os; os.chdir('..')
import multiprocessing
from prev_ob_models.Birgiolas2020.fitting import *
import pandas as pd
from multiprocessing import Pool, Process, Queue

/home/justas/anaconda2/envs/p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Could not load NEURONBackend


# Define the list of isolated cell models to validate

In [2]:
# MC
mcs = [
    'prev_ob_models.BhallaBower1993.isolated_cells.MC',
    'prev_ob_models.Chen2002.isolated_cells.MC',
    'prev_ob_models.David2008.isolated_cells.MC',
    'prev_ob_models.Davison2000.isolated_cells.MC',
    'prev_ob_models.Djurisic2008.isolated_cells.MC',
    'prev_ob_models.KaplanLansner2014.isolated_cells.MC',
    'prev_ob_models.LiCleland2013.isolated_cells.MC',
    'prev_ob_models.McTavish2012.isolated_cells.MC',
    'prev_ob_models.Migliore2005GJs.isolated_cells.MC',
    'prev_ob_models.Migliore2007columns.isolated_cells.MC',
    'prev_ob_models.Migliore2008microcircuits.isolated_cells.MC',
    'prev_ob_models.Migliore2014bulb3d.isolated_cells.MC',
    'prev_ob_models.Migliore2015operators.isolated_cells.MC',
    'prev_ob_models.MiglioreMcTavish2013.isolated_cells.MC',
    'prev_ob_models.Oconnor2012.isolated_cells.MC1',
    'prev_ob_models.Oconnor2012.isolated_cells.MC2',
    'prev_ob_models.Oconnor2012.isolated_cells.MC3',
    'prev_ob_models.Oconnor2012.isolated_cells.MC4',
    'prev_ob_models.Oconnor2012.isolated_cells.MC5',
    'prev_ob_models.Oconnor2012.isolated_cells.MC6',
    'prev_ob_models.RubinCleland2006.isolated_cells.MC',
    'prev_ob_models.Saghatelyan2005.isolated_cells.MC',
    'prev_ob_models.Shen1999.isolated_cells.MC',
    'prev_ob_models.Short2016.isolated_cells.MC',
    'prev_ob_models.Yu2012.isolated_cells.MC',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC1',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC2',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC3',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC4',
    'prev_ob_models.Birgiolas2020.isolated_cells.MC5',
]

#TC
tcs = [
    'prev_ob_models.Birgiolas2020.isolated_cells.TC1',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC2',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC3',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC4',
    'prev_ob_models.Birgiolas2020.isolated_cells.TC5',
]

#GC
gcs = [
    'prev_ob_models.Davison2003.isolated_cells.GC',
    'prev_ob_models.KaplanLansner2014.isolated_cells.GC',
    'prev_ob_models.LiCleland2013.isolated_cells.GC',
    'prev_ob_models.McTavish2012.isolated_cells.GC',
    'prev_ob_models.Migliore2008microcircuits.isolated_cells.GC',
    'prev_ob_models.Migliore2014bulb3d.isolated_cells.GC',
    'prev_ob_models.Migliore2015operators.isolated_cells.GC',
    'prev_ob_models.MiglioreMcTavish2013.isolated_cells.GC',
    'prev_ob_models.Saghatelyan2005.isolated_cells.GC',
    'prev_ob_models.Short2016.isolated_cells.GC',
    'prev_ob_models.Yu2012.isolated_cells.GC',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC1',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC2',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC3',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC4',
    'prev_ob_models.Birgiolas2020.isolated_cells.GC5'
]

debug = [
    'prev_ob_models.Davison2003.isolated_cells.GC',
]

# Use the CellFitter class to compute validation scores

In [3]:
def score(q, type, model, clear_fitter_cache):
    fitter = CellFitter(cell_type=type, fitting_model_class=model)
    
    if clear_fitter_cache:
        fitter.clear_cache()
        
    fitter.best = []
    df, score = fitter.get_best_score()
    
    print("%s SCORE: %s"%(fitter.fitting_model_class, score["model_score"]))
    print(df.transpose())
    
    q.put([df, score])

# In separate processes, run the validation suite against each model

We're using separate processes to ensure a fresh NEURON instance for each model

In [4]:
# type: one of 'tc', 'gc', or 'mc'
# array of isolated cell class names e.g. ['prev_ob_models.Chen2002.isolated_cells.MC']
def build_report(type, models, clear_fitter_cache=False):    
    results = []
    
    for model in models:
        q = Queue()
        p = Process(target=score, args=(q, type, model, clear_fitter_cache))
        p.start()
        results.append(q.get())
        p.join()
        
        print('---------------------------------------------------------------')
    
    report = pd.concat([result[0] for result in results]).transpose()

    csv_file = 'all_%s_model_validation_results.csv'%type
    report.to_csv(csv_file)
    print('Saved report to: %s'%os.path.abspath(csv_file))
    
    return report

# Build the validation results report

Path to the CSV file with the results is printed after all models have been validated

In [ ]:
#build_report('tc', debug, clear_fitter_cache=True)

In [7]:
build_report('gc', gcs, clear_fitter_cache=True)

In [ ]:
build_report('mc', mcs, clear_fitter_cache=False)

In [ ]:
build_report('tc', tcs, clear_fitter_cache=False)